In [1]:
#FIRST of all, install SSMS (SQL Server MicroSoft)
#Second of all, open the SQLQuery1.sql file in SqlServer
#Thrid of all, highlight ONLY the first command (create database Brokerage) then click Execute
#Fourth of all, highlight the rest of the file lines then click Execute.
#by now you should have the database and schema set and done

#now you will need to populate it with data

#Fifth of all, install pyodbc (nice library btw)

#you will then want to connect your database to the ODBC:
#   1) open SQL Server
#   2) right click on the very first element in object explorer (which is your device obvs)
#   3) choose connect
#   4) copy the server name
#   5) paste it in the server string below


#Sixth of all, just run everything below and enjoy :)

#OPTIONAL : you don't need to know how to generate the report file, it is so ez though
#   just open SSMS
#   Select the database "Brokerage"
#   Right click and select Reports then Standard Reports
#   Select any of the reports than seems interesting

In [1]:
import pandas as pd
import pyodbc
import random

In [10]:
#DELETE THIS AFTER EXECUTION



Address_df = pd.read_csv('Street_Address_Listing.csv')
while (len(Address_df) < 1000000):
    Address_df = Address_df.append(Address_df)

Address_df.to_csv('Address.csv')

In [12]:
#DELETE THIS AFTER EXECUTION

names_df = pd.read_csv('NationalNames.csv')
while (len(names_df) < 1000000):
    names_df = names_df.append(names_df)

names_df.to_csv('Names.csv')

Department_df = pd.read_csv('2019_free_title_data.csv')
while(len(Department_df)<1000000):
    Department_df = Department_df.append(Department_df)
Department_df.to_csv('Jobs.csv')

In [2]:
#Helper Functions



alphabet = list('abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ')
alphabetdig = list('abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ123456789')


#to return a random Department Name
def GenerateRandName(digits):
    Random_dept_name = []
    randlength = random.choice(range(2,30))
    while len(Random_dept_name) < randlength:
        if(digits):
            Random_dept_name.append(random.choice(alphabetdig))
        else:
            Random_dept_name.append(random.choice(alphabet))
    return ''.join(map(str,Random_dept_name))

def GenerateRandomDate():
    randDay = random.choice(range(1,29))
    randMonth = random.choice(range(1,12))
    randYear = random.choice(range(2000,2023))
    return pd.to_datetime(str(randDay) + "/"+str(randMonth)+"/"+str(randYear))


In [3]:
names_df = pd.read_csv('Names.csv')
Address_df = pd.read_csv('Address.csv')
df = pd.read_csv('Employee.csv')
Project_df = pd.read_csv('Project.csv')
DeptLoc_df = pd.read_csv('Dept_Locations.csv')

location_pd = pd.read_csv('Address.csv')

Contract_df = pd.read_csv('Contracts.csv')
jobs_df = pd.read_csv('Jobs.csv')



In [3]:
for driver in pyodbc.drivers():
    print(driver)

SQL Server
SQL Server Native Client 11.0
SQL Server Native Client RDA 11.0
ODBC Driver 17 for SQL Server
Microsoft Access Driver (*.mdb, *.accdb)
Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)
Microsoft Access Text Driver (*.txt, *.csv)


In [2]:
df = pd.read_csv('MOCK_DATA.csv')
#Columns in the CSV : first_name Minit last_name Address gender BDate Salary
#attributes in our database: Fname Minit Lname Bdate Address_ Sex Salary
#print(df['first_name'])

#for i in range(1000):
#    df.loc[i,'Minit'] = random.choice(alphabet)
#df.to_csv('MOCK_DATA.csv')

#print(df.iloc[:,:].values.tolist())


In [20]:

df = pd.DataFrame()
for i in range(300):
    df.loc[i, 'first_name'] = names_df.loc[i%1000000, 'Name']
    df.loc[i, 'Minit'] = alphabet[i%27]
    df.loc[i, 'last_name'] = names_df.loc[(i+1)%1000000, 'Name']
    df.loc[i, 'Address'] = Address_df.loc[i%1000000, 'FULL ADDRESS']

    df.loc[i, 'BDate'] = pd.to_datetime(str((i%29) + 1) + "/"+str((i%12) + 1)+"/"+str(2000 + i%40))
    df.loc[i, 'Sex'] = i%3
    df.loc[i, 'Salary'] = i*3000

    if(i%100000 == 0):
        print(i)

df.to_csv('Employee.csv')

0


KeyboardInterrupt: 

In [12]:
#QUERIES TO FILL "Employee"


server = 'LAPTOP-66BOPJMV\SQLEXPRESS'
database = 'Brokerage_Firm'

connection = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};\
                            server='+server+';\
                            database='+database+';\
                            trusted_connection=yes;')
cursor = connection.cursor()


insert_to_employee = "INSERT INTO Employee (Fname, Minit, Lname, Address_, Bdate, Sex, Salary) VALUES(?,?,?, ?, ?, ?, ?)"
select_all_from_employee = "select * from Employee"



for i in range(1000):
    #lst = [df['first_name'], df['Minit'], df['last_name'], pd.to_datetime(df['BDate']), df['Address'], df['gender'].astype(int), df['Salary'].astype(int)]

    #batch size is 3000
    lst = df.iloc[(i)*1000: (i+1)*1000,1:].values.tolist()
    cursor.executemany(insert_to_employee, lst)

connection.commit()



get_Employees_count = "select count (*) from Employee"

cursor = connection.cursor()
count = cursor.execute(get_Employees_count)
print(list(count))
#for row in cursor:
#    print(row)

[(1113331, )]


In [66]:
''' Department Attributes
Dname nvarchar(50),
Dnumber int identity (1,1) Primary Key NOT NULL,
Mgr_SSN int foreign key references Employee(SSN),
Mgr_Start_Date DATE
'''
Dept_df = pd.DataFrame()
Dept_df['Dname'] = GenerateRandName(True)

Dept_df['Mgr_SSN'] = random.choice(range(1,30))
Dept_df['Mgr_Start_Date'] = pd.to_datetime('11/11/2000')

for i in range(1,10000):
    Dept_df.loc[i, 'Dname'] = GenerateRandName(True)
    Dept_df.loc[i, 'Mgr_SSN'] = random.choice(range(1,200000))
   
    Dept_df.loc[i, 'Mgr_Start_Date'] = GenerateRandomDate()


while(len(Dept_df) < 1000000):
    Dept_df = Dept_df.append(Dept_df)

Dept_df.to_csv('Department.csv')



server = 'LAPTOP-66BOPJMV\SQLEXPRESS'
database = 'Brokerage_Firm'

connection = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};\
                            server='+server+';\
                            database='+database+';\
                            trusted_connection=yes;')
cursor = connection.cursor()


insert_to_Department= "INSERT INTO Department (Dname, Mgr_SSN, Mgr_Start_Date) VALUES(?,?,?)"
select_all_from_Department = "select * from Department"
get_Department_count = "select count (*) from Department"

for i in range(1000):
    lst = Dept_df.iloc[i*1000:(i+1)*1000,:].values.tolist()
    cursor.executemany(insert_to_Department, lst)

connection.commit()

count = cursor.execute(get_Department_count)
print(list(count))

Error: ('HY000', '[HY000] [Microsoft][ODBC Driver 17 for SQL Server]Connection is busy with results for another command (0) (SQLExecDirectW)')

In [ ]:
'''

create table Project
(
Pname nvarchar(50),
Pnumber INT unique identity (1,1) NOT NULL,
Plocation nvarchar(50) NOT NULL,
DNO INT foreign key references Department(Dnumber),
CONSTRAINT Project_KEY_COMPOSITE PRIMARY KEY (Pnumber, Plocation)
);
'''

Project_df = pd.DataFrame()
Project_df.loc[0,'Pname'] = 'advanced databases'

Project_df.loc[0,'Plocation'] = Address_df.loc[0, "FULL ADDRESS"]
Project_df.loc[0,'DNO'] = 1


for i in range(1, 10000):
    Project_df.loc[i,'Pname'] = GenerateRandName(True)
    Project_df.loc[i,'Plocation'] = Address_df.loc[i, "FULL ADDRESS"]
    Project_df.loc[i,'DNO'] = i+1
Project_df.head()

while(len(Project_df) < 1000000):
    Project_df = Project_df.append(Project_df)

Project_df.to_csv('Project.csv')

server = 'LAPTOP-66BOPJMV\SQLEXPRESS'
database = 'Brokerage_Firm'

connection = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};\
                            server='+server+';\
                            database='+database+';\
                            trusted_connection=yes;')
cursor = connection.cursor()


insert_to_Project = "INSERT INTO Project (Pname, Plocation, DNO) VALUES(?,?, ?)"
select_all_from_Project = "select * from Project"
get_Project_count = "select count (*) from Project"


for i in range(800):
    lst = Project_df.iloc[i*1000:(i+1)*1000,:].values.tolist()
    cursor.executemany(insert_to_Project, lst)

connection.commit()

count = cursor.execute(get_Project_count)
print(list(count))

[(1608000, )]


In [11]:
''' DEPT LOCATION

create table Dept_Locations
(
Dnumber INT foreign key references Department(Dnumber),
Dlocation nvarchar(50) NOT NULL,
CONSTRAINT Department_KEY_COMPOSITE PRIMARY KEY (Dnumber, Dlocation)
);
'''
Address_df = pd.read_csv('Address.csv')
DeptLoc_df = pd.DataFrame()

#Make it a MILLION
for i in range(10000):
    DeptLoc_df.loc[i,'Dnumber'] = i
    
    DeptLoc_df.loc[i,'Dlocation'] = Address_df.loc[random.choice(range(1,400000)), 'FULL ADDRESS']

DeptLoc_df.to_csv('Dept_Locations.csv')




server = 'LAPTOP-66BOPJMV\SQLEXPRESS'
database = 'Brokerage_Firm'

connection = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};\
                            server='+server+';\
                            database='+database+';\
                            trusted_connection=yes;')
cursor = connection.cursor()


insert_to_DeptLocation = "INSERT INTO Dept_Locations (Dnumber, Dlocation) VALUES(?,?)"
select_all_from_DeptLocations = "select * from Dept_Locations"
get_Dept_location_count = "select count (*) from Dept_Locations"

for i in range(1,8):
    lst = DeptLoc_df.iloc[i*1000:(i+1)*1000,:].values.tolist()
    #try:
    cursor.executemany(insert_to_DeptLocation, lst)
    #except:
    #    print('error')
        #print(lst)




#lst = DeptLoc_df.iloc[1:3,:].values.tolist()
#cursor.executemany(insert_to_DeptLocation, lst)
connection.commit()

count = cursor.execute(get_Dept_location_count)
print(list(count))

SQL Server
SQL Server Native Client 11.0
SQL Server Native Client RDA 11.0
ODBC Driver 17 for SQL Server
Microsoft Access Driver (*.mdb, *.accdb)
Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)
Microsoft Access Text Driver (*.txt, *.csv)


In [22]:
#Testing by inserting only one row into employees


lst = [df['first_name'][0], df['Minit'][0], df['last_name'][0], pd.to_datetime(df['BDate'][0]), df['Address'][0], int(df['gender'][0]), int(df['Salary'][0])]

server = 'LAPTOP-66BOPJMV\SQLEXPRESS'
database = 'Brokerage_Firm'

connection = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};\
                            server='+server+';\
                            database='+database+';\
                            trusted_connection=yes;')
cursor = connection.cursor()
cursor.execute("INSERT INTO Employee (Fname, Minit, Lname, Bdate, Address_, Sex, Salary) VALUES(?,?,?, ?, ?, ?, ?)", lst)

In [4]:
'''
TODO: add a table for "Interns" with attibutes: SSN, Fname, Minit, Lname, Bdate, Address, Major, Projectnum, ManagerSSN
TODO: fill the other tables dummyly
TODO: add table "Contracts" with attributes: Name_Company, Date_created, ProjectID, Product, Price, Date_of_Contract, Emp_Name,Emp_SSN,Contractor_Name, SSN, Delivery_Location

'''



In [56]:
'''
create table Contracts
(
Name_Company nvarchar(50),
Date_created DATE,
ProjectID int,
Product nvarchar(50),
Price int,
Date_of_Contract DATE,
Emp_Name nvarchar(50),
Emp_SSN int,
Contractor_Name nvarchar(50),
Cont_SSN int,
Delivery_Location nvarchar(50),
CONSTRAINT Contracts_KEY_COMPOSITE PRIMARY KEY (Product, Emp_SSN, Cont_SSN)
);
'''
names_df = pd.read_csv('Names.csv')
location_pd = pd.read_csv('Address.csv')
Contract_df = pd.DataFrame()

Contract_df = pd.DataFrame.from_dict({'Name_Company':["ARAMEX"],            
                                        'Date_created': [pd.to_datetime('11/11/2000')],      
                                        'ProjectID': [1],
                                        'Product': ["cement"],
                                        'Price': [2000],
                                        'Date_of_Contract': [pd.to_datetime('11/11/2000')],
                                        'Emp_Name': ["Ayman"],
                                        'Emp_SSN': [1],
                                        'Contractor_Name': ["Moahemd"],
                                        'Cont_SSN': [2],
                                        'Delivery_Location': ["Helwan"]})

j=1

# MAKE IT A MILLION
for i in range(1,10000):
    '''
    Contract_df.insert(i, 'Name_Company', GenerateRandName(False))
    Contract_df.insert(i, 'ProjectID', i/30000)
    Contract_df.insert(i, "Product", 'Prod_'+GenerateRandName(True))
    Contract_df.insert(i, "Price",  random.choice(range(1,2000000)))
    Contract_df.insert(i, "Emp_Name",  names_df.loc[i, 'Name'])
    Contract_df.insert(i, "Emp_SSN", i)
    Contract_df.insert(i, "Contractor_Name", names_df.loc[i-1, 'Name'])
    Contract_df.insert(i, "Cont_SSN", 4000000+i)
    Contract_df.insert(i, "Delivery_Location", location_pd.loc[i, 'FULL ADDRESS'])
    Contract_df.insert(i, "Date_created", GenerateRandomDate())
    Contract_df.insert(i, "Date_of_Contract", GenerateRandomDate())
    '''
    if(i%10000 == 0):
        j = i+ random.choice([1,2,3,4,5,6,7,8,9])

    Contract_df.loc[i, 'Name_Company'] = GenerateRandName(False)
    Contract_df.loc[i, 'ProjectID'] = j%30000
    Contract_df.loc[i, 'Product'] = 'Prod_'+(GenerateRandName(True))

    Contract_df.loc[i, 'Price'] = random.choice(range(1,20000))
    Contract_df.loc[i, 'Emp_Name'] = names_df.loc[i, 'Name']
    Contract_df.loc[i, 'Emp_SSN'] = j
    Contract_df.loc[i, 'Contractor_Name'] = names_df.loc[i-1, 'Name']
    Contract_df.loc[i, 'Cont_SSN'] = 40000+i
    Contract_df.loc[i, 'Delivery_Location'] = location_pd.loc[i, 'FULL ADDRESS']
    
    Contract_df.loc[i, 'Date_created'] = GenerateRandomDate()
    Contract_df['Date_of_Contract'] = GenerateRandomDate()
    j = j+1

Contract_df.to_csv('Contracts.csv')




10000
Name_Company                                 LD
Date_created                         2011-11-26
ProjectID                                9999.0
Product                        Prod_wdhD9KxtvaC
Price                                   17691.0
Date_of_Contract                     2002-10-23
Emp_Name                                Lindsay
Emp_SSN                                  9999.0
Contractor_Name                         Julious
Cont_SSN                                49999.0
Delivery_Location    11843 BRICKSOME AVE, STE D
Name: 9999, dtype: object


In [69]:
server = 'LAPTOP-66BOPJMV\SQLEXPRESS'
database = 'Brokerage_Firm'

connection = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};\
                            server='+server+';\
                            database='+database+';\
                            trusted_connection=yes;')
cursor = connection.cursor()


cursor.execute("delete from Contracts")
cursor.commit()

In [77]:
server = 'LAPTOP-66BOPJMV\SQLEXPRESS'
database = 'Brokerage_Firm'

connection = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};\
                            server='+server+';\
                            database='+database+';\
                            trusted_connection=yes;')
cursor = connection.cursor()




insert_to_Contracts= "INSERT INTO Contracts (Name_Company, Date_created, ProjectID, Product, Price, Date_of_Contract, Emp_Name, Emp_SSN, Contractor_Name, Cont_SSN, Delivery_Location) VALUES(?,?,?, ?,?,?,?,?,?,?,?)"
select_all_from_Contracts = "select * from Contracts"
get_Contracts_count = "select count (*) from Contracts"


for i in range(8):
    lst = Contract_df.iloc[i*1000:(i+1)*1000,1:].values.tolist()
    #try:
    cursor.executemany(insert_to_Contracts, lst)
    #except:
    #    print("hello")
    #    print(Contract_df.iloc[i,1:12])

connection.commit()

count = cursor.execute(get_Contracts_count)
print(list(count))

[(8000, )]


[(1108000, )]


[(7702, )]


In [278]:
import math
''' Intern TABLE ATTRIBUTES
(
SSN int identity (3000000,1) primary key Not NULL,
Fname nvarchar(50),
Minit varchar,
Lname nvarchar(50),
Bdate DATE,
Address_ nvarchar(50),
Sex tinyint check(Sex between 0 and 2),
Salary int,
Major nvarchar(50),
PNO INT foreign key references Project(Pnumber),
Super_SSN int foreign key references Employee(SSN)
);
'''



#new empty dataframe
df_intern = pd.DataFrame()
#df_intern['SSN']=         auto incremented
df_intern['Fname']= ['Ayman', 'Mohamed']
df_intern['Minit']= ['M', 'R']
df_intern['Lname']= ['Reda', 'Abdo']
df_intern['Bdate']= [pd.to_datetime("20/12/2004"), pd.to_datetime("11/11/2000")]
df_intern['Address_']= [Address_df.loc[1, 'FULL ADDRESS'], Address_df.loc[3, 'FULL ADDRESS']]
df_intern['Sex']= [1,1]
df_intern['Salary']= [2000,3000]
df_intern['Major']= ["Computer", "Engineering"]
df_intern['PNO']= [3, 5]                                                                #MUST BE PRESENT IN THE PNO
df_intern['Super_SSN']= [1, 2]

for i in range(300):
    #df_intern.loc[i, 'PNO'] = 600000
    if(i == 3):
        print(type(i))
    df_intern.loc[i, 'Fname'] = names_df.loc[i, 'Name']
    
    df_intern.loc[i, 'Minit'] = random.choice(alphabet)
    df_intern.loc[i, 'Lname'] = names_df.loc[i, 'Name']
    df_intern.loc[i, 'Address_'] = Address_df.loc[i, 'FULL ADDRESS']
    
    randSex = random.choice(range(0,3))
    randSalary = random.choice(range(1, 100000))
    
    df_intern.loc[i, 'Bdate'] = GenerateRandomDate()
    df_intern.loc[i, 'Sex'] = randSex
    df_intern.loc[i, 'Salary'] = randSalary
    df_intern.loc[i, 'Major'] = jobs_df.loc[i%5000, 'top related titles 19']
                                                                                      #Specify we have 60000 project
    df_intern.loc[i, 'Super_SSN'] = random.randint(1, 5000)*100                                  #the first 500000 employees are intern supervisors
    if(i%1000 == 0):
        print(i)


for i in range(300):
    df_intern.loc[i, 'PNO'] = i

while(len(df_intern) < 10000):
    df_intern = df_intern.append(df_intern)
'''
df_intern["Salary"] = int(df_intern["Salary"])
df_intern["Sex"] = int(df_intern["Sex"])

df_intern["Super_SSN"] = int(df_intern["Super_SSN"])
'''
df_intern["PNO"] = (df_intern["PNO"]).astype(int)

df_intern.to_csv('Intern.csv')

0
<class 'int'>


In [4]:
df_intern = pd.read_csv('Intern.csv')
df_intern.head()

,Unnamed: 0,Fname,Minit,Lname,Bdate,Address_,Sex,Salary,Major,PNO,Super_SSN
0,0,Mary,f,Mary,2003-06-13,"7353 HIGHLAND RD, STE B 282",2.0,81694.0,general manager,0,156600.0
1,1,Anna,C,Anna,2000-01-25,"9007 HIGHLAND RD, STE 9",1.0,57209.0,sales manager,1,220600.0
2,2,Emma,P,Emma,2001-02-24,"5830 S SHERWOOD FOREST BLVD, STE A6",2.0,83730.0,principal,2,390500.0
3,3,Elizabeth,g,Elizabeth,2000-04-01,"4520 S SHERWOOD FOREST BLVD, STE 103",2.0,82711.0,owner,3,79200.0
4,4,Minnie,y,Minnie,2000-03-27,"8334 O'HARA CT, STE D",2.0,53267.0,chief executive offi,4,3700.0


In [5]:
server = 'LAPTOP-66BOPJMV\SQLEXPRESS'
database = 'Brokerage_Firm'

connection = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};\
                            server='+server+';\
                            database='+database+';\
                            trusted_connection=yes;')
cursor = connection.cursor()


count = cursor.execute("delete from Intern")
cursor.commit()

In [6]:

server = 'LAPTOP-66BOPJMV\SQLEXPRESS'
database = 'Brokerage_Firm'

connection = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};\
                            server='+server+';\
                            database='+database+';\
                            trusted_connection=yes;')
cursor = connection.cursor()


insert_to_Intern = "INSERT INTO Intern (Fname, Minit, Lname, Bdate, Address_, Sex, Salary, Major, PNO, Super_SSN) VALUES(?,?,?, ?, ?, ?, ?, ?, ?, ?)"
select_all_from_Intern = "select * from Intern"
get_Intern_count = "select count (*) from Intern"

for i in range(1,2000):
    #lst = df_intern.iloc[i,1:].values.tolist()
    lst = [df_intern.iloc[i,1], df_intern.iloc[i,2], df_intern.iloc[i,3], df_intern.iloc[i,4], df_intern.iloc[i,5], int(df_intern.iloc[i,6]), int(df_intern.iloc[i,7]), \
        df_intern.iloc[i,8], int(df_intern.iloc[i,9]), int(df_intern.iloc[i,10])]
    #try:
    cursor.execute(insert_to_Intern, lst)
    #except:
    #    print(df_intern.iloc[i,5:])


connection.commit()

count = cursor.execute(get_Intern_count)
print(list(count))

IntegrityError: ('23000', '[23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]The INSERT statement conflicted with the FOREIGN KEY constraint "FK__Intern__PNO__4316F928". The conflict occurred in database "Brokerage_firm", table "dbo.Project", column \'Pnumber\'. (547) (SQLExecDirectW); [23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]The statement has been terminated. (3621)')

In [195]:
df_intern.iloc[525,1:]

Fname                   Emmer
Minit                       k
Lname                   Emmer
Bdate              2003-05-26
Address_     4068 HIGHLAND RD
Sex                       2.0
Salary                47576.0
Major                     NaN
PNO                     525.0
Super_SSN            280112.0
Name: 525, dtype: object

[(808000, )]
